In [2]:
# libary import 
from gensim import models
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import sys, os, re, csv, codecs
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import multi_gpu_model
import argparse
import konlpy
import re
import os
from konlpy.tag import Okt, Komoran, Twitter, Mecab, Kkma
%matplotlib inline
print('konlpy version = {}'.format(konlpy.__version__))
print('tensorflow version = {}'.format(tf.__version__))

konlpy version = 0.5.2
tensorflow version = 1.14.0


In [3]:
dataset_pos = pd.read_csv('moddle_done_pos_35162.csv', index_col = False, encoding = 'euc-kr')

In [4]:
dataset_neg = pd.read_csv('middle_done_neg_27347.csv', index_col = False, encoding = 'euc-kr')

In [5]:
dataset_pos.drop(columns = ['eva_scr_val'], inplace = True)
dataset_pos['eva_scr_val'] = 1

In [6]:
dataset_neg.drop(columns = ['eva_scr_val'], inplace = True)
dataset_neg['eva_scr_val'] = 0

In [7]:
dataset_pos = dataset_pos[:27347]

In [8]:
round(len(dataset_pos)*0.9)

24612

In [9]:
train_pos = dataset_pos[:round(len(dataset_pos)*0.9)]
test_pos = dataset_pos[round(len(dataset_pos)*0.9):]

In [10]:
train_neg = dataset_neg[:round(len(dataset_neg)*0.9)]
test_neg = dataset_neg[round(len(dataset_neg)*0.9):]

In [11]:
train = pd.concat([train_pos, train_neg], axis =0)
test = pd.concat([test_pos, test_neg], axis = 0)

In [12]:
train.columns = ['id','filtered','contents','rank']
train = train[['id','contents','rank']]
test.columns = ['id','filtered','contents','rank']
test = test[['id','contents','rank']]

In [13]:
train_data = train.sample(frac=1).reset_index(drop=True)
test_data = test.sample(frac=1).reset_index(drop=True)

In [14]:
#re
train_data['contents'] = train_data['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['contents'] = test_data['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [15]:
import warnings

In [16]:
import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

Using TensorFlow backend.


In [17]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
from keras_radam import RAdam

In [20]:
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS=2
LR=1e-5

pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = 'contents'
LABEL_COLUMN = "rank"

In [24]:
train_data.drop(train_data[train_data.contents == ' '].index, inplace=True)
train_data.drop(train_data[train_data.contents == '  '].index, inplace=True)
train_data.drop(train_data[train_data.contents == '   '].index, inplace=True)
train_data.drop(train_data[train_data.contents == '    '].index, inplace=True)
train_data.drop(train_data[train_data.contents == '     '].index, inplace=True)
train_data.drop(train_data[train_data.contents == '      '].index, inplace=True)
train_data.drop(train_data[train_data.contents == '       '].index, inplace=True)
train_data.drop(train_data[train_data.contents == '        '].index, inplace=True)

In [26]:
train_data.reset_index(inplace = True, drop = True)

In [27]:
test_data.drop(test_data[test_data.contents == ' '].index, inplace=True)
test_data.drop(test_data[test_data.contents == '  '].index, inplace=True)
test_data.drop(test_data[test_data.contents == '   '].index, inplace=True)
test_data.drop(test_data[test_data.contents == '    '].index, inplace=True)
test_data.drop(test_data[test_data.contents == '     '].index, inplace=True)
test_data.drop(test_data[test_data.contents == '      '].index, inplace=True)
test_data.drop(test_data[test_data.contents == '       '].index, inplace=True)
test_data.drop(test_data[test_data.contents == '        '].index, inplace=True)

In [28]:
test_data.reset_index(inplace = True, drop = True)

In [29]:
okt = Komoran()
stopwords = ['']
s = time.time()
x_train = []
for i, sentence in enumerate(train_data.contents):
    temp_X = []
    temp_X = list(okt.morphs(sentence)) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    x_train.append(temp_X)

In [31]:
tmp_train = []
for sentence in x_train:
    tmp_x = ' '.join(sentence)
    tmp_train.append(tmp_x)

In [33]:
train_data.contents = tmp_train

In [35]:
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
          token = token.replace("_","")
          token = "##" + token
        token_dict[token] = len(token_dict)

In [36]:
class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [37]:
tokenizer = inherit_Tokenizer(token_dict)

In [42]:
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i])
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [43]:
train_x, train_y = load_data(train_data)
test_x, test_y = load_data(test_data)

100%|██████████| 5470/5470 [00:01<00:00, 3464.96it/s]


In [45]:
def sentence_convert_data(data):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data))):
        print(tokenizer.tokenize(data[i]))
        ids, segments = tokenizer.encode(data[i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def sentence_load_data(sentences):#sentence는 List로 받는다
           
    data_x = sentence_convert_data(sentences)

    return data_x

In [47]:
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

In [48]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 128)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 128)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [49]:
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output


  outputs = keras.layers.Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)



  bert_model = keras.models.Model(inputs, outputs)
  bert_model.compile(
      optimizer=RAdam(learning_rate=0.00001, weight_decay=0.0025),
      loss='binary_crossentropy',
      metrics=['accuracy'])
  
  return bert_model

In [51]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs=10, batch_size=16, verbose = 1, validation_data=(test_x, test_y), shuffle=True)


Train on 49221 samples, validate on 5470 samples
Epoch 1/10
 8720/49221 [====>.........................] - ETA: 5:38:59 - loss: 0.4397 - accuracy: 0.7905

KeyboardInterrupt: 